In [1]:
import torch
import torch.nn as nn

# Implemented by myself
from config import *
from data_processer import CSCDataset, load_confusion, split_torch_dataset
from models import CombineBertModel, DecoderBaseRNN, DecoderTransformer, Trainer
from opencc import OpenCC
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer

/root/anaconda3/envs/simple_csc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained(checkpoint)

In [3]:
converter = OpenCC("t2s")

In [4]:
train_dataset = CSCDataset(
    [SIGHAN_train_dir_err, SIGHAN_train_dir_corr], tokenizer, converter
)
test_dataset = CSCDataset(
    [SIGHAN_train_dir_err14, SIGHAN_train_dir_corr14], tokenizer, converter
)

preprocessing sighan dataset: 2339it [00:00, 10266.85it/s]
preprocessing sighan dataset: 100%|██████████| 2339/2339 [00:00<00:00, 10218.49it/s]


共2339句，共73264字，最长的句子有171字


preprocessing sighan dataset: 3437it [00:00, 6757.74it/s]
preprocessing sighan dataset: 100%|██████████| 3437/3437 [00:00<00:00, 6624.68it/s]

共3437句，共170330字，最长的句子有258字


In [5]:
# split data
train_data, dev_data = split_torch_dataset(train_dataset, 0.3)

train_data_loader = DataLoader(train_data, num_workers=4, shuffle=True, batch_size=16)
dev_data_loader = DataLoader(dev_data, num_workers=4, shuffle=True, batch_size=16)
test_data_loader = DataLoader(test_dataset, num_workers=4, shuffle=True, batch_size=32)

In [6]:
confusion_set = load_confusion(confusion_set_path, tokenizer)

In [7]:
# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

encoder_model = BertModel.from_pretrained(checkpoint)
decoder_model = DecoderBaseRNN(
    model=nn.LSTM,
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)
model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model, confusion_set=confusion_set)

optimizer = AdamW(model.parameters(), lr=learning_rate)
trainer = Trainer(model=model, tokenizer=tokenizer, optimizer=optimizer)

In [9]:
epochs = 35

trainer.train(
    dataloader=train_data_loader, epoch=epochs, test_dataloader=dev_data_loader
)
trainer.test(test_data_loader)

dev Epoch:1/1: 100%|██████████| 44/44 [00:07<00:00,  5.50it/s, batches loss=1.062]

1.002017307010564 {'over_corr': 1895, 'total_err': 882, 'true_corr': tensor(195, device='cuda:0')} {'over_corr': 544, 'total_err': 681, 'true_corr': 34}



dev Epoch:1/1: 100%|██████████| 44/44 [00:07<00:00,  5.57it/s, batches loss=1.148]

0.9422040608796206 {'over_corr': 1744, 'total_err': 882, 'true_corr': tensor(204, device='cuda:0')} {'over_corr': 515, 'total_err': 681, 'true_corr': 41}



dev Epoch:1/1: 100%|██████████| 44/44 [00:07<00:00,  5.50it/s, batches loss=0.669]

...

0.4458432810550386 {'over_corr': 407, 'total_err': 882, 'true_corr': tensor(268, device='cuda:0')} {'over_corr': 231, 'total_err': 681, 'true_corr': 138}



dev Epoch:1/1: 100%|██████████| 44/44 [00:08<00:00,  5.19it/s, batches loss=0.240]

0.44602563977241516 {'over_corr': 392, 'total_err': 882, 'true_corr': tensor(259, device='cuda:0')} {'over_corr': 226, 'total_err': 681, 'true_corr': 139}



train Epoch:35/35:  11%|█         | 11/103 [00:03<00:29,  3.16it/s, batches loss=0.052]


KeyboardInterrupt: 

In [ ]:
# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

encoder_model = BertModel.from_pretrained(checkpoint)
decoder_model = DecoderBaseRNN(
    model=nn.GRU,
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)
model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

optimizer = AdamW(model.parameters(), lr=learning_rate)
trainer = Trainer(model=model, tokenizer=tokenizer, optimizer=optimizer)

In [ ]:
trainer.train(
    dataloader=train_data_loader, epoch=epochs, test_dataloader=dev_data_loader
)
trainer.test(test_data_loader)

In [ ]:
nhead = 2
num_encoder_layers = 2
num_decoder_layers = 2

encoder_model = BertModel.from_pretrained(checkpoint)
decoder_model = DecoderTransformer(
    input_size=encoder_model.config.hidden_size,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
)
model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

optimizer = AdamW(model.parameters(), lr=learning_rate)
trainer = Trainer(model=model, tokenizer=tokenizer, optimizer=optimizer)

In [ ]:
trainer.train(
    dataloader=train_data_loader, epoch=epochs, test_dataloader=dev_data_loader
)
trainer.test(test_data_loader)